In [ ]:
import os
import re
import sys

import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics as skmetrics
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.font_manager import fontManager
from IPython.display import display
from ipywidgets import interact, FloatSlider
from umap import UMAP
import ipynbname
from endaaman.ml import fix_global_seed

fix_global_seed()

project_root = os.path.abspath(os.path.join(os.path.dirname(ipynbname.path()), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
os.chdir(project_root)
print('now working', project_root)


sns.set_theme(style='ticks', palette='tab10')
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 16,
    'figure.dpi': 300,
})

os.makedirs('out/figs', exist_ok=True)


In [ ]:
from PIL import ImageDraw, ImageFont

def get_samples(target, targets, subdir='typical'):
    data = []
    for i, (fold, diag, name, x, y) in enumerate(targets):
        if target == 'cv':
            pp = {
                'Original': ['org', f'data/tmp/cam_crop/{fold}_test/{diag}/{name}.jpg'],
                'UNI(LP)': ['uni_lp', f'out/compare/LMGAOB/fold5_{fold}/frozen_uni_500/pred_crop/test/{diag}/{name}.jpg'],
                'UNI(FT)': ['uni_ft', f'out/compare/LMGAOB/fold5_{fold}/unfrozen_uni_500/pred_crop/test/{diag}/{name}.jpg'],
                r'ViT-L$\mathrm{_{IN}}$(FT)': ['vit_ft', f'out/compare/LMGAOB/fold5_{fold}/unfrozen_baseline-vit_500/pred_crop/test/{diag}/{name}.jpg'],
            }
        elif target == 'ebrains':
            pp = {
                'Original': ['org', f'data/tmp/cam_crop/{fold}_ebrains/{diag}/{name}.jpg'],
                'UNI(LP)': ['uni_lp', f'out/compare/LMGAOB/fold5_{fold}/frozen_uni_500/pred_crop/ebrains/{diag}/{name}.jpg'],
                'UNI(FT)': ['uni_ft', f'out/compare/LMGAOB/fold5_{fold}/unfrozen_uni_500/pred_crop/ebrains/{diag}/{name}.jpg'],
                r'ViT-L$\mathrm{_{IN}}$(FT)': ['vit_ft', f'out/compare/LMGAOB/fold5_{fold}/unfrozen_baseline-vit_500/pred_crop/ebrains/{diag}/{name}.jpg'],
            }
        else:
            raise RuntimeError('Invalid target', target)
        for label, (c, p) in pp.items():
            img = Image.open(p)
            g = 512
            crop_size = g*2
            img = img.crop((x*g, y*g, x*g+crop_size, y*g+crop_size))
            data.append({
                'label': label,
                'diag': diag,
                'image': img,
            })
            dest_dir = f'out/figs/fig4/{subdir}' if target == 'cv' else f'out/figs/fig5/{subdir}'
            os.makedirs(dest_dir, exist_ok=True)
            if c == 'org':                
                draw = ImageDraw.Draw(img)
                bar_w = 224 # equals 100μm
                bar_h = 24
                m = 16
                draw.rectangle(
                    [
                        m,
                        img.height-bar_h-m,
                        m+bar_w,
                        img.height-m,
                    ],
                    fill='black'
                )
            img.save(f'{dest_dir}/{i}_{diag}_{c}.png')
    return data


def show_grid(samples, cols):
    rows = len(samples)//cols
    fig = plt.figure(figsize=(cols*3, rows*3))
    gs = gridspec.GridSpec(rows, cols)
    gs.update(wspace=0.1, hspace=0.2)
    for idx, item in enumerate(samples):
        ax = plt.subplot(gs[idx])
        ax.imshow(item['image'])
        ax.axis('off')
        ax.set_title(f"{item['diag']} - {item['label']}")
        
    # plt.tight_layout()
    plt.show()


In [ ]:
typical = [
    [3, 'G', '19-1132', 0, 4],
    [3, 'G', 'N22-272', 5, 3],
    [3, 'A', 'N22-166', 0, 4],
    [3, 'A', 'N21-163', 3, 1],
    [3, 'O', 'N19-204', 1, 2],
    # ['O', 'N19-204', 5, 3],\
    [3, 'O', '19-1570', 4, 3],
    [3, 'M', 'N21-267', 3, 3],
    [3, 'L', 'N21-297', 5, 2],    
]

typical_samples = get_samples('cv', typical, 'typical')
show_grid(typical_samples, 4)

In [ ]:
atypical = [
    [1, 'G', '22-0107', 2, 1], 
    # [2, 'G', 'N21-205', 5, 2], 
    [4, 'G', 'N21-327', 4, 0], # failed
    [3, 'A', 'N21-163', 3, 5],
    [3, 'A', 'N22-193', 3, 6],
    [1, 'O', 'N19-277', 0, 0],
    [0, 'O', 'N20-276', 2, 2],
    # [1, 'M', '21-2158', 3, 2], # failed
    [4, 'M', 'N20-216', 1, 2], # failed
    [3, 'L', 'N22-263', 6, 4], # failed 各周囲明庭を呈すようなlmyphoma
]

atypical_samples = get_samples('cv', atypical, 'atypical')
show_grid(atypical_samples, 4)

In [ ]:
typical_samples = [
    [0, 'G', 'a1963015-357f-11eb-a1d3-001a7dda7111', 5, 2],
    [0, 'G', 'a1951ea4-357f-11eb-a591-001a7dda7111', 6, 2],    
    [0, 'G', 'a195bade-357f-11eb-96cc-001a7dda7111', 1, 0],   # 分厚い
    [0, 'A', 'a194f78d-357f-11eb-a5e4-001a7dda7111', 3, 1],
    [0, 'A', 'a195e1ef-357f-11eb-b7e2-001a7dda7111', 0, 0],
    [0, 'A', 'a195bb4e-357f-11eb-99dd-001a7dda7111', 1, 1],
    [0, 'A', 'a1982c51-357f-11eb-a788-001a7dda7111', 4, 2],     
    [0, 'O', 'a195e23e-357f-11eb-9b42-001a7dda7111', 1, 1], # AO or 0, 1
    [0, 'O', 'a197ddfd-357f-11eb-836d-001a7dda7111', 1, 0], # AO
    [0, 'O', 'a195bb4a-357f-11eb-adce-001a7dda7111', 1, 1], # O
    [0, 'L', 'a19768d6-357f-11eb-ac19-001a7dda7111', 3, 1], # 小型小型
    [0, 'L', 'a196575f-357f-11eb-b320-001a7dda7111', 3, 1], # 血管周囲
    [0, 'L', 'a1980585-357f-11eb-b797-001a7dda7111', 0, 1], # 隙間出血
    [0, 'L', 'a1971a95-357f-11eb-b88b-001a7dda7111', 0, 0], # パッチをまたいだ一貫した反応
    [0, 'M', 'a1980508-357f-11eb-831f-001a7dda7111', 0, 1], # 必ずしもviableな成分に反応するわけでもない
    [0, 'M', 'a195e22b-357f-11eb-93f0-001a7dda7111', 1, 1], # 壊死の反応パターン
]

typical_samples = get_samples('ebrains', typical_samples, 'typical')
show_grid(typical_samples, 4)

In [ ]:
atypical_samples  = [
    [0, 'G', 'a195bb37-357f-11eb-972f-001a7dda7111', 0, 0], # 間質に反応しながらも一貫してパターンの誤認
    [0, 'G', 'a195bb18-357f-11eb-8a27-001a7dda7111', 2, 2], # 青い
    [0, 'G', 'a195e235-357f-11eb-8350-001a7dda7111', 0, 0], # 青い
    [0, 'A', 'a196cc6b-357f-11eb-9ea0-001a7dda7111', 1, 1], # 青い
    [0, 'A', 'a195e1df-357f-11eb-a578-001a7dda7111', 3, 1], # GBM IDHmut
    [0, 'A', 'a195bacd-357f-11eb-bd17-001a7dda7111', 4, 2], # GとAで揺れた
    [0, 'A', 'a195bacd-357f-11eb-bd17-001a7dda7111', 4, 2], # GとAで揺れた
    [0, 'A', 'a195e242-357f-11eb-a382-001a7dda7111', 1, 1], # DA
    [0, 'A', 'a194f78d-357f-11eb-a5e4-001a7dda7111', 3, 0], # DA
    [0, 'O', 'a196a560-357f-11eb-aa24-001a7dda7111', 1, 1], # AO as L
    [0, 'O', 'a196f36f-357f-11eb-a625-001a7dda7111', 5, 2], # AO
    [0, 'O', 'a195bb19-357f-11eb-a220-001a7dda7111', 2, 0], # O, 青い
    [0, 'O', 'a194f782-357f-11eb-b41e-001a7dda7111', 1, 0], # AO
    [0, 'O', 'a195bac9-357f-11eb-839b-001a7dda7111', 1, 1], # どう見てもOっぽいがAになってる    
    [0, 'L', 'a194f78e-357f-11eb-aef1-001a7dda7111', 1, 0],
    [0, 'L', 'a19609a3-357f-11eb-99d4-001a7dda7111', 1, 1], # 一部でなぞのM判定
    [0, 'M', 'a195e206-357f-11eb-b861-001a7dda7111', 2, 0], # Lと誤判定
]

atypical_samples = get_samples('ebrains', atypical_samples, 'atypical')
show_grid(atypical_samples, 4)